In [1]:
import pandas as pd
import geopandas as gpd
from shapely import geometry

In [2]:
reg_path = "Regions_iledefrance.geojson"
dept_path = "Departements_iledefrance.geojson"
arrs_path = "Arrondissements_iledefrance.geojson"
parisarrs_path = "Arrondissements_paris.geojson"
comm_path = "Communes_iledefrance.geojson"
iris_path = "IRIS_iledefrance.geojson"

reg_gdf = gpd.read_file(reg_path)
dept_gdf = gpd.read_file(dept_path)
arrs_gdf = gpd.read_file(arrs_path)
parisarrs_gdf = gpd.read_file(parisarrs_path)
comm_gdf = gpd.read_file(comm_path)
iris_gdf = gpd.read_file(iris_path)

In [3]:
print(parisarrs_gdf.columns)
print(comm_gdf.columns)

Index(['ID', 'FID', 'NOM', 'NOM_M', 'INSEE_ARM', 'INSEE_COM', 'POPULATION',
       'geometry'],
      dtype='object')
Index(['ID', 'STATUT', 'INSEE_COM', 'INSEE_ARR', 'INSEE_DEP', 'INSEE_REG',
       'POPULATION', 'FID', 'NOM', 'NOM_M', 'INSEE_CAN', 'SIREN_EPCI',
       'geometry'],
      dtype='object')


In [4]:
parisarrs_comm_aligned = parisarrs_gdf.copy()

# add any missing columns that exist in comm_gdf but not in parisarrs_gdf
for col in comm_gdf.columns:
    if col not in parisarrs_comm_aligned.columns:
        parisarrs_comm_aligned[col] = pd.NA

# reorder to match comm_gdf exactly
parisarrs_comm_aligned = parisarrs_comm_aligned[comm_gdf.columns]

print(parisarrs_comm_aligned.columns)

Index(['ID', 'STATUT', 'INSEE_COM', 'INSEE_ARR', 'INSEE_DEP', 'INSEE_REG',
       'POPULATION', 'FID', 'NOM', 'NOM_M', 'INSEE_CAN', 'SIREN_EPCI',
       'geometry'],
      dtype='object')


In [5]:
# Paris department and region codes
parisarrs_comm_aligned["INSEE_DEP"] = "75"
parisarrs_comm_aligned["INSEE_REG"] = "11"

# Arrondissement code comes from INSEE_ARM if available
if "INSEE_ARM" in parisarrs_gdf.columns:
    parisarrs_comm_aligned["INSEE_ARR"] = parisarrs_gdf["INSEE_ARM"]

# For any missing arrondissement code, fill with NA explicitly
parisarrs_comm_aligned["INSEE_ARR"] = parisarrs_comm_aligned["INSEE_ARR"].fillna(pd.NA)

# Optional: set STATUT to "arrondissement"
parisarrs_comm_aligned["STATUT"] = "arrondissement"

# check unique values of INSEE_DEP and STATUT to confirm
print(parisarrs_comm_aligned["INSEE_DEP"].unique())
print(parisarrs_comm_aligned["STATUT"].unique())

['75']
['arrondissement']


In [6]:
comm_gdf_no_paris = comm_gdf[comm_gdf["INSEE_DEP"] != "75"].copy()
print(comm_gdf_no_paris.shape)

(1408, 13)


In [7]:
comm_gdf_merged = gpd.GeoDataFrame(pd.concat([comm_gdf_no_paris, parisarrs_comm_aligned], ignore_index=True), crs=comm_gdf.crs)

print(comm_gdf_merged.shape)

(1428, 13)


In [8]:
comm_gdf_merged.to_file("Communes_iledefrance_with_parisarrs.geojson", driver="GeoJSON")